In [1]:
import scSpace
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import numpy as np
import scipy.io
import scipy.linalg
import sklearn.metrics
import pandas as pd
import random
import torch
from tqdm import tqdm
from scipy.stats import pearsonr
from scipy.spatial.distance import squareform, pdist
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False

In [3]:
load_dir = '~/workspace/scSpace/data/DLPFC_new/data/'
save_dir = '~/workspace/scSpace/data/DLPFC_new/result/'

In [4]:
pearson_list = []

In [5]:
# slice 151507, 151508, 151509, 151510
train_list = ['151507', '151508', '151509', '151510']
test_list = [['151508', '151509', '151510'], ['151507', '151509', '151510'], ['151507', '151508', '151510'], ['151507', '151508', '151509']]
for i in range(4):
    train_data = load_dir + train_list[i] + '_data.csv'
    train_meta = load_dir + train_list[i] + '_meta.csv'
    test_all = test_list[i]
    
    for j in range(3):
        test_data = load_dir + test_all[j] + '_data.csv'
        test_meta = load_dir + test_all[j] + '_meta.csv'
        
        print('train slice:', train_list[i], '; test slice:', test_all[j])
        
        sc_obj, st_obj = scSpace.load_data(sc_data_path=test_data, sc_meta_path=test_meta, st_data_path=train_data, st_meta_path=train_meta, spatial_key=['imagerow','imagecol'])
        sc_obj_hvg, st_obj_hvg = scSpace.preporcess(sc_adata=sc_obj, st_adata=st_obj, st_type='spot', n_features=2000, normalize=True, select_hvg='union')
        
        setup_seed(123)
        starttime = time.time()
        sc_obj_hvg, st_obj_hvg = scSpace.construct_pseudo_space(
            sc_adata=sc_obj_hvg,
            st_adata=st_obj_hvg,
            batch_size=16,
            activation='sigmoid',
            lr=0.001,
            epoch_num=1000,
            log_epoch=1000)
        
        endtime = time.time()
        print(endtime - starttime)
        
        pseudo_dist = scSpace.cal_dist(sc_obj_hvg.obsm['pseudo_space'])
        real_dist = scSpace.cal_dist(np.array(sc_obj_hvg.obs[['imagerow', 'imagecol']]))
        pearson = round(pearsonr(pseudo_dist, real_dist)[0], 3)
        print('PCC = ', pearson)

        pseudo_space_dir = save_dir + test_all[j] + '_pseudo_space(' + 'trained_on_' + train_list[i] + ').csv'
        pd.DataFrame(sc_obj_hvg.obsm['pseudo_space']).to_csv(pseudo_space_dir)
        
        pcc_res = [train_list[i], test_all[j], pearson]
        pearson_list.append(pcc_res)
    

train slice: 151507 ; test slice: 151508
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [03:46<00:00,  4.41it/s]


Epoch [1000/1000]: Batch loss=3126.72705078125
Encoder for source domain training finished.
596.115380525589


100%|██████████| 4381/4381 [00:22<00:00, 196.12it/s] 


PCC =  0.592
train slice: 151507 ; test slice: 151509
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


...

PCC =  0.434
train slice: 151510 ; test slice: 151509
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [04:10<00:00,  3.99it/s]


Epoch [1000/1000]: Batch loss=5266.83154296875
Encoder for source domain training finished.
442.69712495803833


100%|██████████| 4788/4788 [00:26<00:00, 179.79it/s] 


PCC =  0.492


In [6]:
# slice 151669, 151670, 151671, 151672
train_list = ['151669', '151670', '151671', '151672']
test_list = [['151670', '151671', '151672'], ['151669', '151671', '151672'], ['151669', '151670', '151672'], ['151669', '151670', '151671']]
for i in range(4):
    train_data = load_dir + train_list[i] + '_data.csv'
    train_meta = load_dir + train_list[i] + '_meta.csv'
    test_all = test_list[i]
    
    for j in range(3):
        test_data = load_dir + test_all[j] + '_data.csv'
        test_meta = load_dir + test_all[j] + '_meta.csv'
        
        print('train slice:', train_list[i], '; test slice:', test_all[j])
        
        sc_obj, st_obj = scSpace.load_data(sc_data_path=test_data, sc_meta_path=test_meta, st_data_path=train_data, st_meta_path=train_meta, spatial_key=['imagerow','imagecol'])
        sc_obj_hvg, st_obj_hvg = scSpace.preporcess(sc_adata=sc_obj, st_adata=st_obj, st_type='spot', n_features=2000, normalize=True, select_hvg='union')
        
        setup_seed(123)
        starttime = time.time()
        sc_obj_hvg, st_obj_hvg = scSpace.construct_pseudo_space(
            sc_adata=sc_obj_hvg,
            st_adata=st_obj_hvg,
            batch_size=16,
            activation='sigmoid',
            lr=0.001,
            epoch_num=1000,
            log_epoch=1000)
        
        endtime = time.time()
        print(endtime - starttime)
        
        pseudo_dist = scSpace.cal_dist(sc_obj_hvg.obsm['pseudo_space'])
        real_dist = scSpace.cal_dist(np.array(sc_obj_hvg.obs[['imagerow', 'imagecol']]))
        pearson = round(pearsonr(pseudo_dist, real_dist)[0], 3)
        print('PCC = ', pearson)

        pseudo_space_dir = save_dir + test_all[j] + '_pseudo_space(' + 'trained_on_' + train_list[i] + ').csv'
        pd.DataFrame(sc_obj_hvg.obsm['pseudo_space']).to_csv(pseudo_space_dir)
        
        pcc_res = [train_list[i], test_all[j], pearson]
        pearson_list.append(pcc_res)

train slice: 151669 ; test slice: 151670
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [03:18<00:00,  5.03it/s]


Epoch [1000/1000]: Batch loss=3007.218994140625
Encoder for source domain training finished.
392.80388593673706


100%|██████████| 3484/3484 [00:14<00:00, 246.49it/s] 


PCC =  0.523
train slice: 151669 ; test slice: 151671
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


...

PCC =  0.413
train slice: 151672 ; test slice: 151671
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [03:30<00:00,  4.74it/s]


Epoch [1000/1000]: Batch loss=3480.390380859375
Encoder for source domain training finished.
433.71551394462585


100%|██████████| 4093/4093 [00:19<00:00, 211.21it/s]


PCC =  0.576


In [7]:
# slice 151673, 151674, 151675, 151676
train_list = ['151673', '151674', '151675', '151676']
test_list = [['151674', '151675', '151676'], ['151673', '151675', '151676'], ['151673', '151674', '151676'], ['151673', '151674', '151675']]
for i in range(4):
    train_data = load_dir + train_list[i] + '_data.csv'
    train_meta = load_dir + train_list[i] + '_meta.csv'
    test_all = test_list[i]
    
    for j in range(3):
        test_data = load_dir + test_all[j] + '_data.csv'
        test_meta = load_dir + test_all[j] + '_meta.csv'
        
        print('train slice:', train_list[i], '; test slice:', test_all[j])
        
        sc_obj, st_obj = scSpace.load_data(sc_data_path=test_data, sc_meta_path=test_meta, st_data_path=train_data, st_meta_path=train_meta, spatial_key=['imagerow','imagecol'])
        sc_obj_hvg, st_obj_hvg = scSpace.preporcess(sc_adata=sc_obj, st_adata=st_obj, st_type='spot', n_features=2000, normalize=True, select_hvg='union')
        
        setup_seed(123)
        starttime = time.time()
        sc_obj_hvg, st_obj_hvg = scSpace.construct_pseudo_space(
            sc_adata=sc_obj_hvg,
            st_adata=st_obj_hvg,
            batch_size=16,
            activation='sigmoid',
            lr=0.001,
            epoch_num=1000,
            log_epoch=1000)
        
        endtime = time.time()
        print(endtime - starttime)
        
        pseudo_dist = scSpace.cal_dist(sc_obj_hvg.obsm['pseudo_space'])
        real_dist = scSpace.cal_dist(np.array(sc_obj_hvg.obs[['imagerow', 'imagecol']]))
        pearson = round(pearsonr(pseudo_dist, real_dist)[0], 3)
        print('PCC = ', pearson)

        pseudo_space_dir = save_dir + test_all[j] + '_pseudo_space(' + 'trained_on_' + train_list[i] + ').csv'
        pd.DataFrame(sc_obj_hvg.obsm['pseudo_space']).to_csv(pseudo_space_dir)
        
        pcc_res = [train_list[i], test_all[j], pearson]
        pearson_list.append(pcc_res)

train slice: 151673 ; test slice: 151674
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [03:16<00:00,  5.10it/s]


Epoch [1000/1000]: Batch loss=2793.10009765625
Encoder for source domain training finished.
383.5360553264618


100%|██████████| 3635/3635 [00:15<00:00, 237.08it/s] 


PCC =  0.664
train slice: 151673 ; test slice: 151675
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


...

PCC =  0.672
train slice: 151676 ; test slice: 151675
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning Transfer Component Analysis...
Transfer Component Analysis done.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [03:09<00:00,  5.27it/s]


Epoch [1000/1000]: Batch loss=2548.54736328125
Encoder for source domain training finished.
394.9577753543854


100%|██████████| 3566/3566 [00:14<00:00, 242.25it/s]


PCC =  0.573


In [8]:
pd.DataFrame(pearson_list).to_csv('~/workspace/scSpace/data/DLPFC_new/result/pcc_res.csv')